In [6]:
!pip install tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 32.8 MB/s eta 0:00:00


In [10]:
import tenseal as ts
from typing import List

In [22]:
class HEClient:
    def __init__(self):
        # Setup context
        self.context = ts.context(
            ts.SCHEME_TYPE.BFV,
            poly_modulus_degree=8192,
            plain_modulus=1032193
        )
        self.context.generate_galois_keys()
        self.context.generate_relin_keys()

        # Keep reference to secret key
        self.secret_key = self.context.secret_key()

        # Create a public-only context for the cloud
        self.public_context = self.context.copy()
        self.public_context.make_context_public()

    def get_public_context(self) -> ts.Context:
        return self.public_context

    def encrypt_data(self, data: List[int]) -> ts.BFVVector:
        return ts.bfv_vector(self.context, data)

    def decrypt_value(self, encrypted_value: ts.BFVVector) -> float:
        return encrypted_value.decrypt(self.secret_key)[0]


class EncryptedCloudService:
    def __init__(self, cloud_context: ts.Context):
        self.context = cloud_context

    def compute_sum(self, encrypted_vector: ts.BFVVector) -> ts.BFVVector:
        return encrypted_vector.sum()

    def compute_average_decrypted(self, encrypted_vector: ts.BFVVector, count: int, client: HEClient) -> float:
        enc_sum = encrypted_vector.sum()
        total = client.decrypt_value(enc_sum)
        return total / count

In [23]:
# ---------------- MAIN FUNCTION ---------------- #

def main():
    client = HEClient()
    cloud = EncryptedCloudService(client.get_public_context())

    data = [45000, 52000, 61000, 70000, 80000]
    print(f"Original Data: {data}")

    encrypted_data = client.encrypt_data(data)

    encrypted_sum = cloud.compute_sum(encrypted_data)
    decrypted_sum = client.decrypt_value(encrypted_sum)

    decrypted_avg = cloud.compute_average_decrypted(encrypted_data, len(data), client)

    print(f"Decrypted Sum: {decrypted_sum}")
    print(f"Decrypted Average: {decrypted_avg:.2f}")

main()

Original Data: [45000, 52000, 61000, 70000, 80000]
Decrypted Sum: 308000
Decrypted Average: 61600.00
